
#### 手順
- Bigquery環境構築
- データセット作成
- テーブル作成

In [11]:
import sys
from google.cloud import bigquery


print('python version: ', sys.version)
print('bigquery version: ', bigquery.__version__)

python version:  3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]
bigquery version:  1.25.0


In [4]:
import os

# Credentialsの設定
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'project_folder/credential-gbqtoaml-1c3df6d8f54e.json'

# Credentialsの確認
print('Credentials from environ: {}'.format(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))

Credentials from environ: project_folder/credential-gbqtoaml-1c3df6d8f54e.json


#### Dataset作成

In [6]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()
dataset_name = "dataset_cancer"

# TODO(developer): Set dataset_id to the ID of the dataset to create.
dataset_id = "{}.{}".format(client.project, dataset_name)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset bap-gbqtoaml-dev.dataset_cancer


#### bigquery console

bigquery datasetの確認

https://console.cloud.google.com/bigquery


#### Bingqueryのテーブル作成

In [16]:
from google.cloud import bigquery

client = bigquery.Client()
filename = '../data/breast_cancer.csv'
table_id = 'tb_breast_cancer'

dataset_ref = client.dataset(dataset_name)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

Loaded 569 rows into bap-gbqtoaml-dev.dataset_cancer:tb_breast_cancer.


#### data 確認

In [18]:
import pandas
from google.cloud import bigquery

project_name =  "bap-gbqtoaml-dev"
from_table = project_name + "." + dataset_name + "." +table_id

client = bigquery.Client()

sql = """
    SELECT *
    FROM `""" + from_table +"""`
"""

bigquery_data = client.query(sql)

In [19]:
data = bigquery_data.to_dataframe()
data.head(2)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
